In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,70316
2,Huelva,Confirmados PDIA 14 días,8051
3,Huelva,Tasa PDIA 14 días,"1568,8758111347117"
4,Huelva,Confirmados PDIA 7 días,3356
5,Huelva,Tasa PDIA 7 dias,"653,974316503303"
6,Huelva,Total Confirmados,70550
7,Huelva,Curados,54165
8,Huelva,Fallecidos,435


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  70316.0


/tmp/ipykernel_60130/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  20570.0


/tmp/ipykernel_60130/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_60130/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_60130/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_60130/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 147 personas en los últimos 7 días 

Un positivo PCR cada 58 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,70316.0,8051.0,3356.0,513170.0,653.974317,1568.875811,247.0
Huelva-Costa,40889.0,4315.0,1759.0,289548.0,607.498584,1490.253775,134.0
Huelva (capital),20570.0,2497.0,976.0,143837.0,678.545854,1735.992825,81.0
Sierra de Huelva-Andévalo Central,8790.0,1951.0,903.0,67391.0,1339.941535,2895.045332,71.0
Condado-Campiña,20171.0,1773.0,689.0,156231.0,441.013627,1134.857999,43.0
Lepe,3842.0,263.0,121.0,27880.0,434.002869,943.328551,14.0
Cortegana,654.0,188.0,97.0,4602.0,2107.779226,4085.180356,14.0
Valverde del Camino,1915.0,380.0,185.0,12750.0,1450.980392,2980.392157,13.0
Punta Umbría,1916.0,210.0,107.0,15355.0,696.841420,1367.632693,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),431.0,141.0,53.0,2327.0,2277.610657,6059.303825,8.0
Fuenteheridos,80.0,29.0,11.0,709.0,1551.480959,4090.267983,0.0
Cortegana,654.0,188.0,97.0,4602.0,2107.779226,4085.180356,14.0
Jabugo,269.0,92.0,39.0,2260.0,1725.663717,4070.796460,3.0
Rosal de la Frontera,230.0,68.0,33.0,1697.0,1944.608132,4007.071302,6.0
Nerva,614.0,206.0,88.0,5169.0,1702.456955,3985.296963,6.0
Cumbres Mayores,291.0,62.0,17.0,1749.0,971.983991,3544.882790,0.0
Campillo (El),239.0,66.0,32.0,2024.0,1581.027668,3260.869565,0.0
Puerto Moral,37.0,9.0,7.0,278.0,2517.985612,3237.410072,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Higuera de la Sierra,120.0,12.0,1.0,1291.0,77.459334,929.512006,0.0,0.083333
Villalba del Alcor,566.0,19.0,2.0,3366.0,59.417706,564.468212,0.0,0.105263
San Silvestre de Guzmán,84.0,5.0,1.0,614.0,162.866450,814.332248,0.0,0.200000
Nava (La),15.0,5.0,1.0,258.0,387.596899,1937.984496,0.0,0.200000
Gibraleón,1477.0,211.0,53.0,12737.0,416.110544,1656.591034,3.0,0.251185
Cumbres Mayores,291.0,62.0,17.0,1749.0,971.983991,3544.882790,0.0,0.274194
San Bartolomé de la Torre,502.0,29.0,8.0,3761.0,212.709386,771.071524,0.0,0.275862
Campofrío,100.0,7.0,2.0,713.0,280.504909,981.767181,0.0,0.285714
Beas,524.0,51.0,15.0,4341.0,345.542502,1174.844506,2.0,0.294118
